# 자전거 대여문제 코드 연습

오타 유형 정리

Column 오타일경우
list(data)로 확인 가능

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_path = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Bike-Sharing-Dataset/hour.csv'
rides = pd.read_csv(data_path)

dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)
    
fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)

quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

test_data = data[-21*24:]
val_data = data[-81*24:-21*24]
train_data = data[:-81*24]

target_fields = ['cnt', 'casual', 'registered']
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]
val_features, val_targets = val_data.drop(target_fields, axis=1), val_data[target_fields]
train_features, train_targets = train_data.drop(target_fields, axis=1), train_data[target_fields]

test_features, test_targets = test_features.values, test_targets.values
val_features, val_targets = val_features.values, val_targets.values
train_features, train_targets = train_features.values, train_targets.values

# 우리는 타겟 중 첫번째 컬럼(cnt)만 사용할 것이다.
test_cnt = test_targets[:,0]
val_cnt = val_targets[:,0]
train_cnt = train_targets[:,0]


print(train_targets, train_cnt)


# ----------------------------------------------------

'''
단순한 multivariate regression을 해보자!
'''
import time # 학습시간 측정용
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# 모델 준비
simple_regression = linear_model.LinearRegression()

# 모델 training
# Test셋의 첫번째 컬럼(cnt)만 사용할 것이다. 
start_time = time.clock()
simple_regression.fit(train_features, train_cnt )
end_time = time.clock()

print('----  {0:.5f}sec, training complete  ----'.format(end_time-start_time))

# Training & Validation set에서의 예측값 준비
train_pred, val_pred = simple_regression.predict(train_features), simple_regression.predict(val_features)

# Training & Validation set에서의 성능 확인

print("Mean Squared Error on Training set : {0:.5f}".format(mean_squared_error(train_cnt, train_pred)  ))
print("Mean Squared Error on Validation set : {0:.5f}".format(mean_squared_error(val_cnt, val_pred)  ))

print("R-squared Score on Training set : {0:.5f}".format(r2_score(train_cnt, train_pred)))
print("R-squared Score on Validation set : {0:.5f}".format(r2_score(val_cnt, val_pred)))


[[-0.95631172 -0.662736   -0.93016249]
 [-0.82399838 -0.56132647 -0.80463169]
 [-0.86810283 -0.62217218 -0.83766611]
 ...
 [-0.82399838 -0.6830179  -0.76499039]
 [-0.07973581 -0.662736    0.12033204]
 [ 1.75059879 -0.54104456  2.27417616]] [-0.95631172 -0.82399838 -0.86810283 ... -0.82399838 -0.07973581
  1.75059879]
----  0.06533sec, training complete  ----
Mean Squared Error on Training set : 0.30119
Mean Squared Error on Validation set : 0.46282
R-squared Score on Training set : 0.69078
R-squared Score on Validation set : 0.64129
